# SQL Database Connection Diagnostics

This notebook tests connectivity to Fabric SQL Database and identifies permission issues.

**Steps:**
1. Identify current identity
2. Test database connection
3. Check permissions
4. Provide fix recommendations

## Configuration

Set your database name below (must match exactly, case-sensitive):

In [ ]:
# CHANGE THIS to your SQL Database name
DB_NAME = "db_datalineage"

## Step 1: Identify Current Identity

This shows which identity the notebook is running as.

In [ ]:
import base64
import json

def decode_jwt_payload(token):
    """Decode JWT token payload without verification."""
    try:
        parts = token.split('.')
        if len(parts) != 3:
            return None
        payload = parts[1]
        # Add padding if needed
        payload += '=' * (4 - len(payload) % 4)
        decoded = base64.urlsafe_b64decode(payload)
        return json.loads(decoded)
    except Exception as e:
        return {"error": str(e)}

print("=" * 60)
print("STEP 1: IDENTITY CHECK")
print("=" * 60)

try:
    from notebookutils import mssparkutils
    
    # Get token for SQL Database
    token = mssparkutils.credentials.getToken("https://database.windows.net/")
    print("[OK] Successfully acquired token for SQL Database")
    
    # Decode and show identity
    payload = decode_jwt_payload(token)
    if payload and "error" not in payload:
        identity = payload.get('upn') or payload.get('unique_name') or payload.get('appid') or payload.get('name') or 'Unknown'
        identity_type = 'Service Principal' if payload.get('appid') and not payload.get('upn') else 'User'
        
        print(f"\n>>> IDENTITY: {identity}")
        print(f">>> TYPE: {identity_type}")
        print(f">>> TENANT: {payload.get('tid', 'Unknown')}")
        print(f">>> AUDIENCE: {payload.get('aud', 'Unknown')}")
        
        if payload.get('oid'):
            print(f">>> OBJECT ID: {payload.get('oid')}")
        
        print("\n" + "-" * 60)
        print("SAVE THIS FOR CREATE USER STATEMENT:")
        print("-" * 60)
        if identity_type == 'User':
            print(f"CREATE USER [{identity}] FROM EXTERNAL PROVIDER;")
        else:
            print(f"-- For Service Principal, use the app name or ID:")
            print(f"CREATE USER [{payload.get('appid')}] FROM EXTERNAL PROVIDER;")
    else:
        print(f"[WARN] Could not decode token: {payload}")
        
except Exception as e:
    print(f"[ERROR] Failed to get identity: {e}")

## Step 2: Test Database Connection

Attempts to connect to the SQL Database.

In [ ]:
print("=" * 60)
print("STEP 2: CONNECTION TEST")
print("=" * 60)
print(f"Target database: {DB_NAME}")
print()

connection = None
connection_success = False

try:
    from notebookutils import data
    
    print("Attempting connection (may take 30-60s for cold start)...")
    connection = data.connect_to_artifact(DB_NAME)
    
    # Quick test query
    cursor = connection.cursor()
    cursor.execute("SELECT 1 AS test")
    result = cursor.fetchone()
    
    if result and result[0] == 1:
        print("[OK] Connection successful!")
        connection_success = True
    else:
        print("[WARN] Connected but test query returned unexpected result")
        
except Exception as e:
    error_str = str(e)
    print(f"[ERROR] Connection failed!")
    print(f"\nError details: {error_str[:500]}")
    
    # Diagnose specific errors
    print("\n" + "-" * 60)
    print("DIAGNOSIS:")
    print("-" * 60)
    
    if "18456" in error_str or "token-identified principal" in error_str:
        print(">>> CAUSE: Your identity does not have a database user")
        print(">>> FIX: Run CREATE USER statement (see Step 1 output)")
        print("")
        print("In Fabric Portal:")
        print(f"1. Open '{DB_NAME}' SQL Database")
        print("2. Click 'New Query'")
        print("3. Run the CREATE USER statement from Step 1")
        print("4. Run: ALTER ROLE db_owner ADD MEMBER [your-identity];")
        
    elif "timeout" in error_str.lower():
        print(">>> CAUSE: Database cold start or name mismatch")
        print(">>> FIX: ")
        print(f"  1. Verify '{DB_NAME}' exists in THIS workspace")
        print("  2. Open the database in Fabric Portal to wake it up")
        print("  3. Check the name is EXACTLY correct (case-sensitive)")
        
    elif "not found" in error_str.lower() or "does not exist" in error_str.lower():
        print(">>> CAUSE: Database not found in this workspace")
        print(">>> FIX: ")
        print(f"  1. Notebook must be in SAME workspace as '{DB_NAME}'")
        print("  2. Check database name spelling (case-sensitive)")
        
    else:
        print(">>> CAUSE: Unknown - see error details above")

## Step 3: Check Permissions (if connected)

If connection succeeded, checks what permissions you have.

In [ ]:
print("=" * 60)
print("STEP 3: PERMISSION CHECK")
print("=" * 60)

if not connection_success:
    print("[SKIP] Connection failed - fix connection first")
else:
    try:
        cursor = connection.cursor()
        
        # Check current user
        cursor.execute("SELECT USER_NAME(), SUSER_SNAME()")
        user_info = cursor.fetchone()
        print(f"Database user: {user_info[0]}")
        print(f"Login name: {user_info[1]}")
        
        # Check role memberships
        cursor.execute("""
            SELECT r.name as role_name
            FROM sys.database_role_members rm
            JOIN sys.database_principals r ON rm.role_principal_id = r.principal_id
            JOIN sys.database_principals u ON rm.member_principal_id = u.principal_id
            WHERE u.name = USER_NAME()
        """)
        roles = [row[0] for row in cursor.fetchall()]
        
        print(f"\nRoles: {roles if roles else 'None (public only)'}")
        
        # Check if has required permissions
        has_owner = 'db_owner' in roles
        has_read = 'db_datareader' in roles or has_owner
        has_write = 'db_datawriter' in roles or has_owner
        
        print("\nPermission status:")
        print(f"  db_owner: {'YES' if has_owner else 'NO'}")
        print(f"  Can read: {'YES' if has_read else 'NO'}")
        print(f"  Can write: {'YES' if has_write else 'NO'}")
        
        if not has_owner:
            print("\n[WARN] For LineageParser, db_owner is recommended")
            print(f"Run: ALTER ROLE db_owner ADD MEMBER [{user_info[0]}];")
        else:
            print("\n[OK] You have full database access")
            
    except Exception as e:
        print(f"[ERROR] Permission check failed: {e}")

## Step 4: Check Schema Objects (if connected)

Verifies the required schema and tables exist.

In [ ]:
print("=" * 60)
print("STEP 4: SCHEMA CHECK")
print("=" * 60)

if not connection_success:
    print("[SKIP] Connection failed - fix connection first")
else:
    try:
        cursor = connection.cursor()
        
        # Check schemas
        cursor.execute("SELECT name FROM sys.schemas WHERE name IN ('raw', 'meta')")
        schemas = [row[0] for row in cursor.fetchall()]
        
        print(f"Required schemas: {schemas}")
        
        if 'raw' not in schemas or 'meta' not in schemas:
            print("\n[ERROR] Missing required schemas!")
            print("Run schema-ddl.sql to create the schema")
        else:
            # Check key tables
            cursor.execute("""
                SELECT TABLE_SCHEMA + '.' + TABLE_NAME 
                FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_SCHEMA IN ('raw', 'meta')
                ORDER BY TABLE_SCHEMA, TABLE_NAME
            """)
            tables = [row[0] for row in cursor.fetchall()]
            
            print(f"\nTables found ({len(tables)}):")
            for t in tables:
                print(f"  - {t}")
            
            # Check stored procedures
            cursor.execute("""
                SELECT SCHEMA_NAME(schema_id) + '.' + name 
                FROM sys.procedures 
                WHERE SCHEMA_NAME(schema_id) = 'meta'
            """)
            procs = [row[0] for row in cursor.fetchall()]
            
            print(f"\nStored procedures ({len(procs)}):")
            for p in procs:
                print(f"  - {p}")
                
            if len(tables) >= 5 and len(procs) >= 5:
                print("\n[OK] Schema appears to be set up correctly")
            else:
                print("\n[WARN] Schema may be incomplete - run schema-ddl.sql")
                
    except Exception as e:
        print(f"[ERROR] Schema check failed: {e}")

## Summary

Run all cells above, then check the results.

In [ ]:
print("=" * 60)
print("DIAGNOSTIC SUMMARY")
print("=" * 60)

if connection_success:
    print("[OK] Connection: SUCCESS")
    print("\nYour notebook can connect to the database.")
    print("If LineageParser still fails, check:")
    print("  1. raw.objects has data (run Copy Pipeline first)")
    print("  2. User has db_owner role for write access")
else:
    print("[FAIL] Connection: FAILED")
    print("\nTo fix:")
    print("1. Open the SQL Database in Fabric Portal")
    print("2. Click 'New Query'")
    print("3. Run these commands (replace with YOUR identity from Step 1):")
    print("")
    print("   CREATE USER [your-email@domain.com] FROM EXTERNAL PROVIDER;")
    print("   ALTER ROLE db_owner ADD MEMBER [your-email@domain.com];")
    print("")
    print("4. Re-run this notebook to verify")